In [8]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from census import Census

# Census & gmaps API Keys
from config import (api_key)
c = Census(api_key)


In [17]:
# Retrieve data from census API
census_data = c.acs5.get(("NAME", "B01003_001E", "B05003_008E", "B05003_019E",
                          "B17001_002E", "B02001_002E", "B02001_003E",
                          "B02001_004E", "B02001_005E", "B02001_006E"), 
                         {'for': 'state:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "population",
                                      "B05003_008E": "population_mal_18",
                                      "B05003_019E": "population_fem_18",
                                      "B17001_002E": "poverty_count",
                                      "B02001_002E": "pop_white",
                                      "B02001_003E": "pop_black_afr_american",
                                      "B02001_004E": "pop_american_indian_alaska",
                                      "B02001_005E": "pop_asian",
                                      "B02001_006E": "pop_native_hawaiian_pacific_islands",
                                      "NAME": "state", "state":"number"})

# Add in Population 18+
census_pd["population_18"] = \
    census_pd["population_mal_18"].astype(
        int) + census_pd["population_fem_18"].astype(int)

# Add in Poverty Rate (Poverty Count / Population)
census_pd["poverty_rate"] = 100 * \
    census_pd["poverty_count"].astype(
        int) / census_pd["population"].astype(int)

# Add in Population Race Sum
census_pd["pop_race_sum"] = \
    census_pd["pop_white"].astype(
        int) + census_pd["pop_black_afr_american"].astype(
            int) + census_pd["pop_american_indian_alaska"].astype(
                int) + census_pd["pop_asian"].astype(
                    int) + census_pd["pop_native_hawaiian_pacific_islands"].astype(int)



# Final DataFrame
census_pd = census_pd[["state", "population", "population_mal_18", "population_fem_18", "population_18",
                       "poverty_count", "poverty_rate", "pop_race_sum", "pop_white", "pop_black_afr_american",
                       "pop_american_indian_alaska", "pop_asian", "pop_native_hawaiian_pacific_islands"]]

# Visualize
print(len(census_pd))

census_pd.head()

census_pd.dtypes


52


state                                   object
population                             float64
population_mal_18                      float64
population_fem_18                      float64
population_18                            int32
poverty_count                          float64
poverty_rate                           float64
pop_race_sum                             int32
pop_white                              float64
pop_black_afr_american                 float64
pop_american_indian_alaska             float64
pop_asian                              float64
pop_native_hawaiian_pacific_islands    float64
dtype: object

In [18]:
census_pd['population'] = census_pd['population'].astype(int)
census_pd['population_mal_18'] = census_pd['population_mal_18'].astype(int)
census_pd['population_fem_18'] = census_pd['population_fem_18'].astype(int)
census_pd['poverty_count'] = census_pd['poverty_count'].astype(int)
census_pd['pop_white'] = census_pd['pop_white'].astype(int)
census_pd['pop_black_afr_american'] = census_pd['pop_black_afr_american'].astype(int)
census_pd['pop_american_indian_alaska'] = census_pd['pop_american_indian_alaska'].astype(int)
census_pd['pop_asian'] = census_pd['pop_asian'].astype(int)
census_pd['pop_native_hawaiian_pacific_islands'] = census_pd['pop_native_hawaiian_pacific_islands'].astype(int)


In [19]:
census_pd = census_pd.reset_index()

In [20]:
census_pd.to_csv("census.csv", index=False)



In [6]:
# File to Load (Remember to Change These)
path = "president_county_candidate.csv"

vote_df = pd.read_csv(path)

vote_df

,state,county,candidate,party,total_votes,won
0,Delaware,Kent County,Joe Biden,DEM,44552,True
1,Delaware,Kent County,Donald Trump,REP,41009,False
2,Delaware,Kent County,Jo Jorgensen,LIB,1044,False
3,Delaware,Kent County,Howie Hawkins,GRN,420,False
4,Delaware,New Castle County,Joe Biden,DEM,195034,True
...,...,...,...,...,...,...
31453,Arizona,Maricopa County,Donald Trump,REP,995665,False
31454,Arizona,Maricopa County,Jo Jorgensen,LIB,31705,False
31455,Arizona,Mohave County,Donald Trump,REP,78535,True
31456,Arizona,Mohave County,Joe Biden,DEM,24831,False


In [9]:
path = "election_2020_data2.xlsx"

president_data = pd.read_excel(path, engine="openpyxl")



In [10]:
#president_data.round({'GeoID': 0, 'Trump': 0, 'Biden': 0, 'Other': 0, 'Total': 0})

president_data['WnrPerc'] = president_data['WnrPerc'] * 100




In [11]:
decimals = 2    
president_data['WnrPerc'] = president_data['WnrPerc'].apply(lambda x: round(x, decimals))

In [13]:
president_data = president_data.reset_index()

In [14]:
president_data.to_csv("president_state.csv", index=False)

In [5]:
path = "state.xlsx"

state_data = pd.read_excel(path, engine="openpyxl")


In [3]:
state_data

,state
0,Alabama
1,Alaska
2,Arizona
3,Arkansas
4,California
5,Colorado
6,Connecticut
7,Delaware
8,District of Columbia
9,Florida


In [7]:
state_data.to_csv("state.csv", index=False)

In [22]:
path = "president_county_candidate.csv"

president_county_candidate = pd.read_csv(path)

In [24]:
president_county_candidate.set_index("index", inplace=True)

In [25]:
president_county_candidate

,state,county,candidate,party,total_votes,won
index,,,,,,
0,Delaware,Kent County,Joe Biden,DEM,44552,True
1,Delaware,Kent County,Donald Trump,REP,41009,False
2,Delaware,Kent County,Jo Jorgensen,LIB,1044,False
3,Delaware,Kent County,Howie Hawkins,GRN,420,False
4,Delaware,New Castle County,Joe Biden,DEM,195034,True
...,...,...,...,...,...,...
31453,Arizona,Maricopa County,Donald Trump,REP,995665,False
31454,Arizona,Maricopa County,Jo Jorgensen,LIB,31705,False
31455,Arizona,Mohave County,Donald Trump,REP,78535,True


In [27]:
president_county_candidate.to_csv("president_county_candidate.csv", index=False)